<a href="https://colab.research.google.com/github/fukagai-takuya/gifu-ai/blob/main/gifu-ai-2025-12-21/FLUX1_schnell_GitClone_Diffusers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Google Drive のマウント (Google Drive 側で予め下記のコマンドを実行していると仮定しています。)

git clone https://github.com/huggingface/diffusers.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

2. git clone した diffusers のソースコードを優先して参照するようにします

In [ ]:
import sys
sys.path.insert(0, "/content/drive/MyDrive/diffusers/src")

3. git clone した diffusers のソースコードが優先して参照されることを確認します

In [ ]:
import diffusers
print(diffusers.__file__)

4. diffusers を使用して FLUX.1 [schnell] を実行する際に必要なパッケージをインストールします

In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers accelerate safetensors
!pip install sentencepiece protobuf einops

5. pdb より高機能な Python デバッガ ipdb をインストールします

In [ ]:
!pip install ipdb

6. FLUX.1 [schnell] を使用した Pipeline を用意します

In [ ]:
import torch
from diffusers import FluxPipeline

pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-schnell",
    torch_dtype=torch.float16
).to("cuda")


7. FLUX.1 [schnell] を使用して画像を生成します

In [ ]:
prompt = 'The phrase "Gifu AI Study Group" formed from small colorful geometric shapes.'

image = pipe(
    prompt,
    guidance_scale=0.0,
    num_inference_steps=4,
    max_sequence_length=256
).images[0]

image.save("Gifu-AI-study-group.png")
